In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from aleister_producer  import AleisterFeedAgent

In [ ]:
afa = AleisterFeedAgent()

In [ ]:
afa.setup_data_provider()

In [ ]:
afa.subscribe_realtime_data()

In [ ]:
afa.stop_subscribe_realtime_data()

In [ ]:
afa.init_db()

In [ ]:
data = afa.fetch_hist_realtime_data(20211201,20211211, ['orderbook'])[20211209]['orderbook']

In [ ]:
afa.db_accesser.dao.find_one("orderbook")

In [ ]:
afa.start_realtime_fetch()

In [ ]:
#!/usr/bin/env python
import pika

connection = pika.BlockingConnection(
    pika.ConnectionParameters(host='localhost'))

channel = connection.channel()

channel.queue_declare(queue='aleister')

def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

def on_request(ch, method, props, body):
    self.logger.info(f"[DONE] via MQ. Host={method}, pro:{props}, cd:{ch}")
    n = int(body)

    print(" [.] fib(%s)" % n)
    response = fib(n)

    ch.basic_publish(exchange='',
                     routing_key=props.reply_to,
                     properties=pika.BasicProperties(correlation_id = \
                                                         props.correlation_id),
                     body=str(response))
    ch.basic_ack(delivery_tag=method.delivery_tag)

channel.basic_qos(prefetch_count=1)
channel.basic_consume(queue='aleister', on_message_callback=on_request)

print(" [x] Awaiting RPC requests")
channel.start_consuming()

In [ ]:
import pika,time
class MessageWorker(object):
    def __init__(self):
        self.connection = pika.BlockingConnection(
        pika.ConnectionParameters(host='localhost'))
        self.channel = self.connection.channel()
        self.channel.queue_declare(queue='aleister')

    def on_request(self, ch, method, props, body):
        print('MessageWorker get message ------', body)
        response = self.task(body)
        print('MessageWorker callback on queue ------', props.reply_to)
        ch.basic_publish(exchange='',
                        routing_key=None,
                        properties=pika.BasicProperties(
                            correlation_id=props.correlation_id),
                        body=str(response))
        print('MessageWorker send message to MessageSender')
        ch.basic_ask(delivery_tag=method.delivery_tag)

    def work(self):
        self.channel.basic_qos(prefetch_count=2)
        self.channel.basic_consume(on_message_callback=self.on_request, queue='aleister')
        print('[x]Waiting message...')
        self.channel.start_consuming()


    def task(self,body):
        time.sleep(3)
        return body



worker = MessageWorker()
worker.work()